In [1]:
inColab = False  # check if program is run in Google Colab

if 'google.colab' in str(get_ipython()):
    inColab = True
    from google.colab import files
    from google.colab import drive
    drive.mount('/content/drive')

import pandas as pd
import pickle
import numpy as np
import os
import io
import time
# from redis_cache import cache_it  # caches data into pickle

In [4]:
baseDir = os.getcwd()

# running option
while True:
    try:
        runLabel = str(input('Label for this run: '))
        compModel = input('Compute model instead of loading it: ') in ['y', 'Y', 'yes', 'Yes', 'YES', '1', '0']
        
        if compModel:
            print("New model will be computed")
        else:
            print("Existing model will be loaded")
        
        break
    except ValueError:
        print("Wrong input, please correct your input.")

Label for this run: trial1
Compute model instead of loading it: yes
New model will be computed


## Importing data

In [5]:
# Importing & sorting data
D_X_df = pd.read_csv(baseDir + r'\data\train_features.csv', index_col = 'pid')
D_y_df = pd.read_csv(baseDir + r'\data\train_labels.csv', index_col = 'pid')
D_test_df = pd.read_csv(baseDir + r'\data\test_features.csv', index_col = 'pid')

# sorting rows by increasing index
# D_X_df = D_X_df.sort_index()
# D_y_df = D_y_df.sort_index()
# D_test_df  = D_test_df.sort_index()

# indices_train = list(D_X_df.index.unique())
# indices_test = list(D_test_df.index.unique())

## Aggregating values of same patient and extending feature vector TO DO: loading existing data doesn't work properly

In [6]:
path_X = os.path.join(baseDir,'results','agg_X.csv')
path_test = os.path.join(baseDir,'results','agg_test.csv')

if True: # not (os.path.exists(path_X) and os.path.exists(path_test)):  # only create file if the file doesn't exist yet
    
    print("Computing data ... ")
    # grouping by index
    D_X_df_min = D_X_df.groupby(['pid'],sort=False).min()
    D_test_df_min = D_test_df.groupby(['pid'],sort=False).min()

    D_X_df_mean = D_X_df.groupby(['pid'],sort=False).mean()
    D_test_df_mean = D_test_df.groupby(['pid'],sort=False).mean()

    D_X_df_max = D_X_df.groupby(['pid'],sort=False).max()
    D_test_df_max = D_test_df.groupby(['pid'],sort=False).max()

    D_X_df_diff = D_X_df_max - D_X_df_min
    D_test_df_diff = D_test_df_max - D_test_df_min

    D_X_df = pd.concat([D_X_df_min, D_X_df_mean, D_X_df_max,D_X_df_diff], axis=1, sort=False)
    D_test_df = pd.concat([D_test_df_min,D_test_df_mean, D_test_df_max,D_test_df_diff], axis=1, sort=False)
    
    D_X_df.to_csv(path_X, index=True, header = True, float_format='%.3f') #, compression='zip')
    D_test_df.to_csv(path_test, index=True, header = True, float_format='%.3f') #, compression='zip')
else:
    print("Loading data ... ")
    D_X_df = pd.read_csv(path_X, index_col = 'pid')
    D_X_test = pd.read_csv(path_test, index_col = 'pid')


# Visualizing data
D_X_percNan = np.round(pd.DataFrame(D_X_df.isna().sum()/D_X_df.shape[0]),3).T
display(D_test_df.head(20))
print("Percent nan values: ")
display(D_X_percNan)

Computing data ... 


,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
pid,,,,,,,,,,,,,,,,,,,,,
0,1,39.0,NaN,38.5,17.0,NaN,36.0,9.1,13.0,-9.0,...,10.0,0.0,NaN,4.0,5.1,12.0,4.6,NaN,25.0,0.06
10001,1,62.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,31.0,NaN,NaN,24.0,NaN
10003,1,72.0,NaN,NaN,NaN,2.0,36.0,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,12.0,NaN,NaN,32.0,0.12
10004,1,44.0,NaN,NaN,10.0,NaN,36.0,12.7,NaN,NaN,...,0.0,8.0,NaN,NaN,0.0,14.0,0.0,NaN,37.0,NaN
10005,1,88.0,NaN,39.9,42.0,NaN,36.0,8.8,25.0,NaN,...,NaN,6.0,NaN,0.0,0.0,20.0,NaN,NaN,33.0,NaN
10008,2,59.0,NaN,21.6,39.0,1.4,36.0,12.8,26.0,3.0,...,0.0,4.0,NaN,9.0,0.0,17.0,NaN,NaN,32.0,0.03
10011,1,45.0,NaN,NaN,7.0,NaN,36.0,9.0,NaN,NaN,...,NaN,3.0,NaN,NaN,0.0,10.0,NaN,NaN,36.0,NaN
10017,1,36.0,NaN,23.6,4.0,NaN,36.0,12.1,NaN,NaN,...,0.0,1.0,NaN,NaN,1.8,13.0,0.0,NaN,18.0,NaN
10018,1,65.0,37.0,NaN,23.0,NaN,38.0,8.7,NaN,NaN,...,NaN,6.0,NaN,NaN,0.0,44.0,NaN,NaN,55.0,0.00


Percent nan values: 


,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,0.0,0.0,0.93,0.589,0.265,0.744,0.023,0.264,0.587,0.7,...,0.75,0.001,0.967,0.56,0.229,0.0,0.752,0.853,0.021,0.579


## Imputing data

In [7]:
# imputing nan data - changing to median
D_test_df = D_test_df.fillna(D_test_df.median())
D_X_df = D_X_df.fillna(D_X_df.median())

#Same as :
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='median').fit(D_X_df)
D_X_df = pd.DataFrame(imp_mean.transform(D_X_df), index = D_X_df.index)
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median').fit(D_test_df)
D_test_df = pd.DataFrame(imp_mean.transform(D_test_df), index = D_test_df.index)

# visualizing
print("Train data: ")
display(D_X_df.head())
print("Labels for train data")
display(D_y_df.head())
print("Test data: ")
display(D_test_df.head())

C:\Users\Sebas\anaconda3\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


Train data: 


,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
pid,,,,,,,,,,,,,,,,,,,,,
1,3.0,34.0,29.5,31.0,12.0,1.64,36.0,8.5,24.0,-2.0,...,0.0,0.0,0.0,3.0,2.4,41.0,0.0,0.0,42.0,0.08
10,1.0,71.0,29.5,27.8,12.0,1.64,36.0,14.6,23.0,-2.0,...,0.0,3.0,0.0,0.0,0.0,19.0,0.0,0.0,39.0,0.04
100,2.0,68.0,29.5,20.9,21.0,1.64,35.0,12.5,27.0,-2.0,...,0.0,8.0,0.0,0.0,0.0,37.0,0.0,0.0,61.0,0.04
1000,1.0,79.0,26.0,31.0,22.0,3.66,36.0,9.2,23.0,-2.0,...,0.0,5.0,0.0,0.0,0.0,41.0,0.0,0.0,67.0,0.00
10000,1.0,76.0,29.5,25.7,22.0,1.64,36.0,10.4,25.0,0.0,...,0.0,3.0,0.0,1.0,2.0,26.0,0.0,0.0,57.0,0.07


Labels for train data


,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
pid,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.1,85.4,100.0,59.9
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8,100.6,95.5,85.5
100,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16.5,88.3,96.5,108.1
1000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,19.4,77.2,98.3,80.9
10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.6,76.8,97.7,95.3


Test data: 


,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
pid,,,,,,,,,,,,,,,,,,,,,
0,1.0,39.0,30.0,38.5,17.0,1.63,36.0,9.1,13.0,-9.0,...,10.0,0.0,0.0,4.0,5.1,12.0,4.6,0.0,25.0,0.06
10001,1.0,62.0,30.0,30.8,17.0,1.63,37.0,10.5,24.0,-2.0,...,0.0,5.0,0.0,0.0,0.0,31.0,0.0,0.0,24.0,0.03
10003,1.0,72.0,30.0,30.8,17.0,2.00,36.0,10.5,24.0,-2.0,...,0.0,2.0,0.0,0.0,0.0,12.0,0.0,0.0,32.0,0.12
10004,1.0,44.0,30.0,30.8,10.0,1.63,36.0,12.7,24.0,-2.0,...,0.0,8.0,0.0,0.0,0.0,14.0,0.0,0.0,37.0,0.03
10005,1.0,88.0,30.0,39.9,42.0,1.63,36.0,8.8,25.0,-2.0,...,0.0,6.0,0.0,0.0,0.0,20.0,0.0,0.0,33.0,0.03


In [8]:
# ALTERNATIVE IMPUTING METHOD:

# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imp_mean = IterativeImputer(random_state=0).fit(D_X_df)
# D_X_df = pd.DataFrame(imp_mean.transform(D_X_df),index=D_X_df.index)

# imp_mean = IterativeImputer(random_state=0).fit(D_test_df)
# D_test_df = pd.DataFrame(imp_mean.transform(D_test_df),index=D_test_df.index)

# D_test_df.head(20)

In [9]:
# defining numpy arrays for computing
# features
D_X = np.array(D_X_df)
D_y = np.array(D_y_df)
D_test = np.array(D_test_df)

# labels
D_y_c_df = D_y_df.iloc[:,0:11]
D_y_r_df = D_y_df.iloc[:,11:]

D_y_c = np.array(D_y_c_df)
D_y_r = np.array(D_y_r_df)

print(f'The percentage of administered tests is : \n{np.round(D_y_c_df.mean()*100,1)}')

The percentage of administered tests is : 
LABEL_BaseExcess          26.8
LABEL_Fibrinogen           7.4
LABEL_AST                 24.0
LABEL_Alkalinephos        23.6
LABEL_Bilirubin_total     24.1
LABEL_Lactate             20.0
LABEL_TroponinI           10.0
LABEL_SaO2                23.4
LABEL_Bilirubin_direct     3.4
LABEL_EtCO2                6.6
LABEL_Sepsis               5.7
dtype: float64


In [10]:
# DEPRECATED

# idx_ones = np.where(D_y_c[:,0] == 1)[0]
# idx_zeroes = np.where(D_y_c[:,0] == 0)[0]

# out_vec = np.random.choice(idx_zeroes, len(idx_ones))

# D_x_select = D_X_stand[[out_vec, idx_ones],:]
# D_y_select = D_y[[out_vec, idx_ones],:]

# from sklearn.utils import shuffle
# D_x_select, D_y_select = shuffle(D_x_select, D_y_select, random_state=0)

# # D_y_df['LABEL_ABPm'].target.value_counts()
# print(out_vec.shape)
# print(idx_ones.shape)





# # Importing imputed data
# D_X_df = pd.read_csv(baseDir + 'cleanedData/X_train.csv', header = None)
# D_y_df = pd.read_csv(baseDir + 'cleanedData/y_train.csv', header = None)

# #classification labels
# D_y_df_c = D_y_df[np.arange(11)]
# #regression values
# D_y_df_r = D_y_df[np.arange(11,15)]

# D_X = np.array(D_X_df)
# D_y = np.array(D_y_df)
# D_y_c = np.array(D_y_df_c)
# D_y_r = np.array(D_y_df_r)

# print(f'D_x head with size {D_X_df.shape}: \n{D_X_df.head()} ')
# print(f'D_y_c head with size {D_y_df_c.shape}: \n{D_y_df_c.head()}')
# print(f'D_y_r head with size {D_y_df_r.shape}: \n{D_y_df_r.head()}')

## Normalizing data

In [11]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()

scaler.fit_transform(D_X)
D_X_stand = scaler.transform(D_X)
D_test_stand = scaler.transform(D_test)

# save standardization
name = os.path.join(baseDir, 'results', f'standardized_{runLabel}.csv')
print(f'File was saved under {name}')

pd.DataFrame(D_X_stand).to_csv(name, index=True, header = True, float_format='%.3f') #, compression='zip')

display(pd.DataFrame(D_test_stand).head())

File was saved under C:\Users\Sebas\Documents\ETH_Zurich\2021_Spring\IML\IML-projects\Project_2\results\standardized_trial1.csv


,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,-0.160088,-1.402508,0.219735,0.349183,-0.228271,-0.144938,-0.401101,-0.765758,-3.589828,-2.863013,...,1.226002,-1.044044,-0.032137,2.649333,1.537779,-0.685050,29.772859,-0.050963,-0.736679,0.331745
1,-0.160088,-0.004486,0.219735,-0.182083,-0.228271,-0.144938,0.859057,-0.008376,0.282772,-0.006475,...,-0.051220,0.131623,-0.032137,-0.270078,-0.455239,0.915724,-0.092594,-0.050963,-0.791702,-0.385829
2,-0.160088,0.603349,0.219735,-0.182083,-0.228271,0.317117,-0.401101,-0.008376,0.282772,-0.006475,...,-0.051220,-0.573777,-0.032137,-0.270078,-0.455239,-0.685050,-0.092594,-0.050963,-0.351514,1.766893
3,-0.160088,-1.098590,0.219735,-0.182083,-0.638763,-0.144938,-0.401101,1.181795,0.282772,-0.006475,...,-0.051220,0.837022,-0.032137,-0.270078,-0.455239,-0.516547,-0.092594,-0.050963,-0.076396,-0.385829
4,-0.160088,1.575886,0.219735,0.445777,1.237773,-0.144938,-0.401101,-0.928054,0.634827,-0.006475,...,-0.051220,0.366756,-0.032137,-0.270078,-0.455239,-0.011040,-0.092594,-0.050963,-0.296490,-0.385829


## K-fold data splits

In [12]:
# for regression portion
from sklearn.model_selection import KFold

kf = KFold(3,shuffle=True,random_state=13)

ind = np.arange(D_X.shape[0])

ind_splits = list(kf.split(ind))

In [13]:
# for classification portion
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3, shuffle=True,random_state=13)

ind_splits_skf = [list(skf.split(D_X,D_y[:,i])) for i in range(D_y_c.shape[1])]
# len(ind_splits_skf)

## Downselection of features - optional

In [14]:
# # using nan value thresholds to filter data, this could be a greedy selection however moving forward
# thresh = 1.0
# # print(D_X_percNan<thresh)
# selection = D_X_percNan <= thresh
# D_X = D_X[:,selection]
# D_test = D_test[:,selection]

# selection = selection[0:D_X_stand.shape[1]]
# D_X_stand = D_X_stand[:,selection]
# D_test_stand = D_test_stand[:,selection]

# print(D_X_stand.shape)
# print(D_X.shape)
# print(D_test.shape)
# print(D_test_stand.shape)


## OPTION2 ##
# from sklearn.svm import LinearSVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectFromModel

# lsvc = LinearSVC(C=0.01).fit(D_X_stand,D_y[:,2])
# selector = SelectFromModel(estimator=LogisticRegression(),max_features=4).fit(D_X_stand,D_y[:,3])

# D_X_stand_row2 = selector.transform(D_X_stand)
# D_X_stand_row2.shape

## PCA visualization - optional

In [15]:
# from sklearn.decomposition import PCA
# from matplotlib import pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# def pcaPlotter(X,y, title='pca plotter', first=500):
#   pca = PCA(n_components = 3)
#   X_2d = pca.fit_transform(X)

#   colors = ['#1F77B4', '#FF7F0E']
#   markers = ['o', 's']
#   fig = plt.figure()
#   ax = fig.add_subplot(111, projection='3d')

#   for count, val in enumerate(y):
#     val=bool(val)
#     ax.scatter(X_2d[count,0],X_2d[count,1],X_2d[count,2],marker=markers[val],c=colors[val])

#     if count > first:
#       break

#   plt.title(title)

#   plt.show()

# pcaPlotter(D_X_stand_row2,D_y[:,3],title='Visualize',first=500)

## Scoring - optional

In [16]:
# import sklearn.metrics as metrics

# VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
# TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
#          'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
#          'LABEL_Bilirubin_direct', 'LABEL_EtCO2']


# def get_score(df_true, df_submission):
#     df_submission = df_submission.sort_values('pid')
#     df_true = df_true.sort_values('pid')
#     task1 = np.mean([metrics.roc_auc_score(df_true[entry], df_submission[entry]) for entry in TESTS])
#     task2 = metrics.roc_auc_score(df_true['LABEL_Sepsis'], df_submission['LABEL_Sepsis'])
#     task3 = np.mean([0.5 + 0.5 * np.maximum(0, metrics.r2_score(df_true[entry], df_submission[entry])) for entry in VITALS])
#     score = np.mean([task1, task2, task3])
#     print(task1, task2, task3)
#     return score


# filename = 'sample.zip'
# df_submission = pd.read_csv(filename)

# # generate a baseline based on sample.zip
# df_true = pd.read_csv(filename)
# for label in TESTS + ['LABEL_Sepsis']:
#     # round classification labels
#     df_true[label] = np.around(df_true[label].values)

# print('Score of sample.zip with itself as groundtruth', get_score(df_true, df_submission))

## Train model classification - replaced by new model

In [17]:
# from sklearn import svm
# from sklearn.metrics import classification_report
# from sklearn.metrics import precision_recall_fscore_support

# from sklearn.svm import LinearSVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectFromModel

# from sklearn.utils import shuffle

# import sklearn.metrics as metrics

# # outputs
# bestModels_c = []
# bestSelector_c = []
# bestKFold_c = []
# bestROCScore_c = []

# mult = 1.00
# n_features = 20

# if True: # loadM == 'c':

#   for idx_label in range(D_y_c.shape[1]):

#     # for stratified k-fold
#     ind_splits = ind_splits_skf[idx_label]

#     ## only pick best features

#     selector = SelectFromModel(estimator=LinearSVC(max_iter = 1e7, C=1),max_features=n_features).fit(D_X_stand,D_y[:,idx_label])
#     # plotting PCA selector for each label
#     pcaPlotter(selector.transform(D_X_stand),D_y[:,idx_label],title=str(D_y_c_df.columns[idx_label]),first=200)
#     #####

#     ######## classification per label

#     f1_best = 0
#     bestKFold = 0
#     bestROCScore = 0
#     for idx, (train_index, val_index) in enumerate(ind_splits):

#       D_X_train = D_X_stand[train_index,:]
#       D_y_train = D_y_c[train_index,idx_label]

#       D_X_val = D_X_stand[val_index,:]
#       D_y_val = D_y_c[val_index,idx_label]

#       # print(idx)

#       # balancing data set

#       idx_ones = np.where(D_y_train == 1)[0]
#       idx_zeroes = np.where(D_y_train == 0)[0]

#       # print(idx_zeroes)

#       out_vec = np.random.choice(idx_zeroes, len(idx_ones))

#       D_X_train = D_X_train[np.concatenate((out_vec, idx_ones)),:]
#       D_y_train = D_y_train[np.concatenate((out_vec, idx_ones))]
      
#       D_X_train, D_y_train = shuffle(D_X_train, D_y_train, random_state=0)

#       ####

#       # only best features
#       D_X_train = selector.transform(D_X_train)
#       D_X_val = selector.transform(D_X_val)
#       ####

#       relWeight = round(D_y_train.shape[0]/np.sum(D_y_train),2)
#       print(f'The relative weight of 1 to 0 is {relWeight}')

#       svc = svm.SVC(gamma = 'scale', max_iter = 1e5, tol = 1e-3, decision_function_shape = 'ovo', C = 1, kernel='linear', class_weight='balanced' , verbose=True, probability=False, cache_size=7500) #class_weight = {0:1,1:(mult*relWeight)}
#       model = svc.fit(D_X_train,D_y_train)

#       # validate
#       y_pred = model.predict(D_X_val)
      
#       f1 = precision_recall_fscore_support(y_pred, D_y_val , average='weighted')[2]

#       # receiver curve score
#       w = np.array(model.coef_)
#       b = np.array(model.intercept_)

#       real_prediction = D_X_val.dot(np.transpose(w)) + b
#       roc_score = metrics.roc_auc_score(D_y_val,real_prediction)
#       print(f'For fold {idx} the roc_score is {roc_score}')

#       ####
    
#       if roc_score > bestROCScore:
#         bestROCScore = roc_score
#         bestModel = model
#         bestKFold = idx
#         print(f'\nBest model for label {D_y_c_df.columns[idx_label]} is kfold {idx}. This model has  {np.round(D_y_c_df.mean()*100,1)[idx_label]} % tests')
#         print(classification_report(D_y_val,y_pred))

#     # print(precision_recall_fscore_support(y_pred, D_y_val , average='weighted'))  

#     bestModels_c.append(bestModel)
#     bestSelector_c.append(selector)
#     bestKFold_c.append(bestKFold)
#     bestROCScore_c.append(bestROCScore)
#     ######### end of calssification per label

#   print(f'All the ROC scores are: {bestROCScore_c}')

In [18]:
# print(f'All the ROC scores are: {bestROCScore_c}')
# print(f'The means are are {[np.mean(bestROCScore_c[0:-2]), bestROCScore_c[-1]]}')

In [19]:
# a = np.array([1,2,5,6])
# b = np.array([8,8])
# np.concatenate((a,b)).shape

# Train classification models listed with _c

In [20]:
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
import sklearn.metrics as metrics

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

from sklearn.utils import shuffle

bestModels_c_seps = []
bestSelector_c_seps = []
bestKFold_c_seps = []
bestROCScore_c_seps = []

mult = 1.00
n_features = 60

# getting time when starting training
import datetime
now = datetime.datetime.now()
unixTime = round(now.timestamp())

# create a new model for each label
if compModel: # loadM == 'c':

  for idx_label in range(D_y_c.shape[1]):

    #idx_label = D_y_c.shape[1]-1

    # for stratified k-fold
    ind_splits = ind_splits_skf[idx_label]

    ## only pick best features

#     selector = SelectFromModel(estimator=LinearSVC(max_iter = 1e5, C=1),max_features=n_features).fit(D_X_stand,D_y[:,idx_label])
    #selector = SelectFromModel(estimator=svm.SVC(class_weight='balanced',kernel='linear'),max_features=n_features).fit(D_X_stand,D_y[:,idx_label])
    # plotting PCA selector for each label
    #pcaPlotter(selector.transform(D_X_stand),D_y[:,idx_label],title=str(D_y_c_df.columns[idx_label]),first=200)
    #####

    ######## classification per label

    f1_best = 0
    bestKFold = 0
    bestROCScore = 0
    
    for idx, (train_index, val_index) in enumerate(ind_splits):

      D_X_train = D_X_stand[train_index,:]
      D_y_train = D_y_c[train_index,idx_label]

      D_X_val = D_X_stand[val_index,:]
      D_y_val = D_y_c[val_index,idx_label]

      # print(idx)

      # balancing data set

      idx_ones = np.where(D_y_train == 1)[0]
      idx_zeroes = np.where(D_y_train == 0)[0]

      # print(idx_zeroes)

      out_vec = np.random.choice(idx_zeroes, len(idx_ones))

      D_X_train = D_X_train[np.concatenate((out_vec, idx_ones)),:]
      D_y_train = D_y_train[np.concatenate((out_vec, idx_ones))]
      
      D_X_train, D_y_train = shuffle(D_X_train, D_y_train, random_state=0)

      ####

      # only best features
      #D_X_train = selector.transform(D_X_train)
      #D_X_val = selector.transform(D_X_val)
      ####

      relWeight = round(D_y_train.shape[0]/np.sum(D_y_train),2)
      print(f'The relative weight of 1 to 0 is {relWeight}')

      svc = svm.SVC(gamma = 'scale', max_iter = 1e7, tol = 1e-4, decision_function_shape = 'ovo', C = 2, kernel='rbf', class_weight='balanced' , verbose=True, probability=True, cache_size=7500) #class_weight = {0:1,1:(mult*relWeight)}
      #svr = svm.SVR(gamma = 'scale', C = 0.1)
      
      model = svc.fit(D_X_train,D_y_train)
      #model_r = svr.fit(D_X_train,D_y_train)

      # validate
      y_pred = model.predict(D_X_val)
      
      f1 = precision_recall_fscore_support(y_pred, D_y_val , average='weighted')[2]

      # receiver curve score
      # w = np.array(model.coef_)
      # b = np.array(model.intercept_)

      #real_prediction = D_X_val.dot(np.transpose(w)) + b
      real_prediction = model.predict_proba(D_X_val)[:,1]
      #real_prediction_r = np.clip(model_r.predict(D_X_val),0,1)
      roc_score = metrics.roc_auc_score(D_y_val,real_prediction)
      print(f'\nFor fold {idx} the roc_score is {roc_score}')

      ####
    
      if roc_score > bestROCScore:
        bestROCScore = roc_score
        bestModel = model
        bestKFold = idx
        print(f'\nBest model for label {D_y_c_df.columns[idx_label]} is kfold {idx}. This model has  {np.round(D_y_c_df.mean()*100,1)[idx_label]} % tests')
        print(classification_report(D_y_val,y_pred))

      # print(precision_recall_fscore_support(y_pred, D_y_val , average='weighted'))  

    bestModels_c_seps.append(bestModel)
#     bestSelector_c_seps.append(selector)
    bestKFold_c_seps.append(bestKFold)
    bestROCScore_c_seps.append(bestROCScore)
    ######### end of calssification per label

  print(f'All the ROC scores are: {bestROCScore_c_seps}')

The relative weight of 1 to 0 is 2.0
[LibSVM]
For fold 0 the roc_score is 0.8788571431475226

Best model for label LABEL_BaseExcess is kfold 0. This model has  26.8 % tests
              precision    recall  f1-score   support

         0.0       0.92      0.81      0.87      4633
         1.0       0.62      0.82      0.70      1699

    accuracy                           0.81      6332
   macro avg       0.77      0.82      0.78      6332
weighted avg       0.84      0.81      0.82      6332

The relative weight of 1 to 0 is 2.0
[LibSVM]
For fold 1 the roc_score is 0.8912609301417386

Best model for label LABEL_BaseExcess is kfold 1. This model has  26.8 % tests
              precision    recall  f1-score   support

         0.0       0.93      0.83      0.88      4633
         1.0       0.64      0.82      0.72      1699

    accuracy                           0.83      6332
   macro avg       0.78      0.83      0.80      6332
weighted avg       0.85      0.83      0.84      6332



[LibSVM]
For fold 1 the roc_score is 0.8765292963093009

Best model for label LABEL_EtCO2 is kfold 1. This model has  6.6 % tests
              precision    recall  f1-score   support

         0.0       0.98      0.85      0.91      5914
         1.0       0.26      0.77      0.39       418

    accuracy                           0.84      6332
   macro avg       0.62      0.81      0.65      6332
weighted avg       0.93      0.84      0.88      6332

The relative weight of 1 to 0 is 2.0
[LibSVM]
For fold 2 the roc_score is 0.8849692146976452

Best model for label LABEL_EtCO2 is kfold 2. This model has  6.6 % tests
              precision    recall  f1-score   support

         0.0       0.98      0.86      0.92      5913
         1.0       0.28      0.77      0.41       418

    accuracy                           0.86      6331
   macro avg       0.63      0.81      0.66      6331
weighted avg       0.94      0.86      0.88      6331

The relative weight of 1 to 0 is 2.0
[LibSVM]
For

In [24]:
bestKFold_c_seps.append(bestKFold)
bestROCScore_c_seps.append(bestROCScore)

real_prediction = model.predict_proba(D_X_val)[:,1]

real_prediction

array([0.34848318, 0.3296042 , 0.60397034, ..., 0.80741593, 0.66070177,
       0.33198094])

In [25]:
np.mean([np.mean(bestROCScore_c_seps[0:-2]),bestROCScore_c_seps[-1]])

0.7418924490620658

## Saving classification models

In [35]:
if compModel:
    pickle_out = open(os.path.join(baseDir, 'models', f'{unixTime}_{runLabel}_c'),'wb')
    pickle.dump([bestModels_c_seps], pickle_out)
    pickle_out.close()


# Train regression models listed with _r

In [37]:
# # Evaluation
# bestkFold_c = [0,1,1,1,1,1,1,0,2,1,1]
# D_X_stand[train_index[i][1],:] for i in bestFold_c

In [38]:
# D_X_stand[ind_splits[2][1],:]

array([[-0.16008754,  0.54256564,  0.01174885, ..., -0.05096326,
         0.03365143, -0.14663768],
       [-0.16008754,  0.84648343,  0.01174885, ..., -0.05096326,
         1.02407585,  0.57093625],
       [-0.16008754, -0.67310549,  0.01174885, ..., -0.05096326,
         1.13412301, -0.14663768],
       ...,
       [-0.16008754, -0.24762059,  0.01174885, ..., -0.05096326,
         0.03365143, -1.10340293],
       [-0.16008754,  0.29943142,  0.01174885, ..., -0.05096326,
         0.52886364, -0.14663768],
       [-0.16008754,  1.63666967,  0.01174885, ...,  0.35997984,
         0.47384006, -0.14663768]])

In [ ]:
# pd.DataFrame(D_X_stand)

In [39]:
# D_X_stand.shape

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

bestModels_r = []
bestKFold_r = []
bestR2_score_r = []

from sklearn.linear_model import LinearRegression

if True:

  for idx_label in range(D_y_r.shape[1]):

    ######## classification per label

    score_best = float('-inf')
    for idx, (train_index, val_index) in enumerate(ind_splits):

      D_X_train = D_X_stand[train_index,:]
      D_y_train = D_y_r[train_index,idx_label]

      D_X_val = D_X_stand[val_index,:]
      D_y_val = D_y_r[val_index,idx_label]

      #svr prediction
      svc = svm.SVR(kernel='rbf',verbose=True)
      model = svc.fit(D_X_train,D_y_train)
      score_m = model.score(D_X_val, D_y_val)

      y_pred = model.predict(D_X_val)

      # validate
      # y_pred = model.predict(D_X_val)
      # score_m = model.score(D_X_val, D_y_val)

      # model = LinearRegression().fit(D_X_train, D_y_train)
      # score_m = model.score(D_X_train, D_y_train)
      # score_m = model.score(D_X_val, D_y_val)

      # Rsquared score minimize
      r2 = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(D_y_val, y_pred))
      
      if r2 > score_best:
        score_best = r2
        bestModel = model
        print(f'\nBest model for label {idx_label} is index: {idx} with score {score_best}')

    # # print(precision_recall_fscore_support(y_pred, D_y_val , average='weighted'))  

    bestModels_r.append(bestModel)
    bestR2_score_r.append(score_best)
    ######### end of calssification per label

  print(f'The regression scores are: {bestR2_score_r}')

[LibSVM]
Best model for label 0 is index: 0 with score 0.6704425183757803
[LibSVM]

In [ ]:
# All scores
totalScore = [np.mean(bestROCScore_c_seps[0:-2]), bestROCScore_c_seps[-1], np.mean(bestR2_score_r)]
print(f'The total scores are: {totalScore} with a mean of: {np.mean(totalScore)}')


## Saving regression models

In [ ]:
if compModel:
  pickle_out = open(os.path.join(baseDir, 'models', f'{unixTime}_{runLabel}_r'),"wb")
  pickle.dump(bestModels_r, pickle_out)
  pickle_out.close()

In [ ]:
bestModels_r[1].predict(D_test_stand)

# Data prediction

In [ ]:
# optionally loading models

if not compModel:
  pathC = input('path of classification model: ')
  pathR = input('path of regression model: ')
  
  file = open(pathC, 'rb')
  temp = pickle.load(file)
  bestModels_c = temp[0]
  bestSelector_c = temp[0]
  file.close()

  file = open(pathR, 'rb')
  bestModels_r = pickle.load(file)
  file.close()

In [ ]:
D_test_stand

In [ ]:
np.sum(bestModels_c[0].predict(D_test_stand))/D_test_stand.shape[0]

In [ ]:
# # W_r = [mod.coef_ for mod in bestModels_r]
# # W_r = np.transpose(np.array(W_r))
# # print(f' Shape of W_r: {W_r.shape}')

# W_c = [mod.coef_[0] for mod in bestModels_c]
# W_c = np.transpose(np.array(W_c))
# print(f' Shape of W_c: {W_c.shape}')

# bias = [mod.intercept_[0] for mod in bestModels_c]
# bias = np.transpose(np.array(bias))
# print(f' Shape of bias: {bias.shape}')

In [ ]:

# bias

In [ ]:
# D_test_predict = bestSelector_c[10].transform(D_test_stand)
# pd.DataFrame(D_test_predict)

In [ ]:
# from scipy.special import expit
# certainty = 1

# result_c = [expit(certainty*(bestSelector_c[idx].transform(D_test_stand).dot(val)+bias[idx])) for idx, val in enumerate(np.transpose(W_c))]
# result_c = np.transpose(np.array(result_c))

In [ ]:
# bias[1]

In [ ]:
# from scipy.special import expit
# certainty = 1
# result_c = expit(certainty*(D_test_predict.dot(W_c)+bias))
# print(f'Shape of result_c is: {result_c.shape}')

result_c = [model.predict_proba(D_test_stand)[:,1] for model in bestModels_c]
result_c = np.transpose(np.array(result_c))

#result_r = D_test.dot(W_r)
result_r = [mod.predict(D_test_stand) for mod in bestModels_r]
result_r = np.transpose(np.array(result_r))
# print(f'Shape of result_r is: {result_r.shape}')


In [ ]:
print(result_c.shape)
print(result_r.shape)

## Validating that manual computation is equal to prediction

In [ ]:
# predictions = bestModels_c[0].predict(D_test_stand)
# w = np.transpose(bestModels_c[0].coef_[0])
# bias = bestModels_c[0].intercept_
# calculations = (np.sign(D_test_stand.dot(w) + bias)+1)/2


# sum(predictions == calculations)/calculations.shape[0]

In [ ]:
# (np.sign(D_test_stand.dot(w) + bias)+1)/2

In [ ]:
result_c.shape
# result_r.shape

## putting numpy array back to dataframe

In [ ]:
D_test_df.index
D_y_df.columns

df_c = pd.DataFrame(result_c,index=D_test_df.index)
df_r = pd.DataFrame(result_r,index=D_test_df.index)
df_out = pd.concat([df_c, df_r], axis=1, sort=False)
df_out.columns = D_y_df.columns
df_out

## saving dataframe

In [ ]:
name = f'/content/drive/My Drive/ETHZ/IML/Project_2/results/{unixTime}_result_{runLabel}.csv'
print(f'File was saved under {name}')
df_out.to_csv(name, index=True, header = True, float_format='%.3f') #, compression='zip')

In [ ]:
runLabel

In [ ]:
bestModels_c[0].coef_